In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import random
import scipy


# Local search algorithm

The local search algorithm is a simple metaheuristic algorithm used for solving optimization problems. It is implemented below to solve a typical knapsack problem. The optimization problem solves for 9 decision variables and has two constraints. The knapsack problem's decision variables are binary.

In [17]:
# function to compute objective function, takes argument x a vector of bit operators
def objective(x):
    # compute objective function value given decision variable vector
    z = 8*x[0] + 11*x[1] + 9*x[2] + 12*x[3] + 14*x[4] + 10*x[5] + 6*x[6] + 7*x[7] + 13*x[8]
    return z

# functions to compute constraint values, takes argument x a vector of bit operators

def constraint_1(x):
    # compute constrain value in form g(x) <= 0
    cons = x[0] + 2*x[1]+ 3*x[2] + 2*x[3] + 3*x[4] + 4*x[5] + x[6] + 5*x[7] + 3*x[8] - 16
    
    # is contraint violated? ie does cons <= 0
    if cons <= 0:
        return False # ie constraint not violated
    else:
        return True # ie constrain violated

def constraint_2(x):
    # loop through each decision vector entry to ensure bit operators only
    for i in x:
        if (i == 0) or (i == 1):
            return False # ie entry is a bit operator so contraint NOT violoated
        else: 
            return True # ie an entry is not a bit operator so constraint IS violated


# Local search algorithm help functions

# take decision vector of length n and return n new vectors, each new vector has one different bit vector flipped
def single_bit_complementing_operator(x):
    vect_length = len(x) # vector length determines how many new vectors to generate
    pos_to_flip = 0 # loop counter which will indicate where to flip the bit operator for each new vec
    new_vectors = [] # empty list to store new vectors

    # create n new vectors
    for i in range(vect_length):
        # set new vector equal to original
        new_vect = x.copy() # ensure a true copy is assigned rather than a reference

        # flip relevant position
        if new_vect[pos_to_flip] == 0:
            new_vect[pos_to_flip] = 1
        else:
            new_vect[pos_to_flip] = 0

        # increment which position to flip bit operator
        pos_to_flip += 1

        # save new vector
        new_vectors.append(new_vect)

    return new_vectors

# take number of neighbour vectors, determine each ones objective function value and return neighbour with highest objective function value
def neighbour_selection(neighbours_array):
    num_vects = len(neighbours_array) 
    neighbour_results = pd.DataFrame(columns=['vector','obj_res','constraints'])

    # for each neighbour calc objective function value and check if constrains are fine
    for i in range(num_vects):

        # select a single neighbour
        neighbour_vect = neighbours_array[i]
        
        # first check contraints: constraints return boolean results, so use boolean sum to check if all good
        if constraint_1(neighbour_vect) + constraint_2(neighbour_vect) == 0:
            obj_func_result = objective(neighbour_vect)
            # print(neighbour_vect,'\t',obj_func_result)

            # store results
            neighbour_results = neighbour_results.append({'vector':neighbour_vect,'obj_res':obj_func_result,'constraints':'passed'}, ignore_index=True)

        # else constraint failed therefore no need to calculate objective function results    
        else:
            obj_func_result = objective(neighbour_vect)
            # print(neighbour_vect,'\t','contraints failed')
            # store results
            neighbour_results = neighbour_results.append({'vector':neighbour_vect,'obj_res':obj_func_result,'constraints':'failed'}, ignore_index=True)

        # fix column datatype issues
        neighbour_results['obj_res'] = neighbour_results['obj_res'].astype(float)
    return neighbour_results




def local_search(x):
    s = x # initial decision vector 
    f_s_prev = 1 # store previous best results for stopping criteria
    f_s = objective(s) # initial decision vector objection function value
    iteration = 0 # iteration counter
    max_iterations = 8 # stopping criteria = max number of iterations
    length_descent_walk = 1 # start at one because of initial condition counting

    while iteration <= max_iterations:

        print('Iteration: ', iteration)
        print('Current best: ',s,'\t',' Obj func res: ',f_s)

        # step 1: generate new neighbours
        new_neighbours = single_bit_complementing_operator(s)

        # step 2: evaluate neighbours objective function values + check constraints
        neighbour_results = neighbour_selection(new_neighbours)

        # step 3: select best neighbour
        feasible_neighbours = neighbour_results.loc[neighbour_results['constraints']=='passed'] # only consider neighbours who satisfy both constraints
        best_new_neighbour_index  = feasible_neighbours['obj_res'].idxmax()
        best_new_neighbour = feasible_neighbours.loc[best_new_neighbour_index]['vector']
        best_new_neighbour_obj_res = feasible_neighbours.loc[best_new_neighbour_index]['obj_res']

        # display iteration neighbourhood
        display(neighbour_results)

        # step 4: check if new best is better than previous
        if f_s < best_new_neighbour_obj_res:
            print('New best found!')
            f_s_prev = f_s
            s = best_new_neighbour
            f_s = best_new_neighbour_obj_res
        
        else:
            print('New neighborhood doesn\'t contain improvement, optimum found!')
            print('Optimum: ',s,'\t',' Obj func res: ',f_s)
            iteration = max_iterations + 1

        # step 5: increment iterator
        iteration += 1
        length_descent_walk += 1

    return s,f_s, length_descent_walk


In [5]:
# Initial starting conditions to test
s_0_1 = [0, 1, 1, 1, 0, 0, 1, 1, 1] 

# run local search from above starting position
local_search(s_0_1)

Iteration:  0
Current best:  [0, 1, 1, 1, 0, 0, 1, 1, 1] 	  Obj func res:  58


,vector,obj_res,constraints
0,"[1, 1, 1, 1, 0, 0, 1, 1, 1]",66.0,failed
1,"[0, 0, 1, 1, 0, 0, 1, 1, 1]",47.0,passed
2,"[0, 1, 0, 1, 0, 0, 1, 1, 1]",49.0,passed
3,"[0, 1, 1, 0, 0, 0, 1, 1, 1]",46.0,passed
4,"[0, 1, 1, 1, 1, 0, 1, 1, 1]",72.0,failed
5,"[0, 1, 1, 1, 0, 1, 1, 1, 1]",68.0,failed
6,"[0, 1, 1, 1, 0, 0, 0, 1, 1]",52.0,passed
7,"[0, 1, 1, 1, 0, 0, 1, 0, 1]",51.0,passed
8,"[0, 1, 1, 1, 0, 0, 1, 1, 0]",45.0,passed


New neighborhood doesn't contain improvement, optimum found!
Optimum:  [0, 1, 1, 1, 0, 0, 1, 1, 1] 	  Obj func res:  58


([0, 1, 1, 1, 0, 0, 1, 1, 1], 58, 2)

In [7]:
# Initial starting conditions to test
s_0_2 = [0, 1, 0, 1, 0, 0, 0, 1, 0]

# run local search from above starting position
local_search(s_0_2)

Iteration:  0
Current best:  [0, 1, 0, 1, 0, 0, 0, 1, 0] 	  Obj func res:  30


,vector,obj_res,constraints
0,"[1, 1, 0, 1, 0, 0, 0, 1, 0]",38.0,passed
1,"[0, 0, 0, 1, 0, 0, 0, 1, 0]",19.0,passed
2,"[0, 1, 1, 1, 0, 0, 0, 1, 0]",39.0,passed
3,"[0, 1, 0, 0, 0, 0, 0, 1, 0]",18.0,passed
4,"[0, 1, 0, 1, 1, 0, 0, 1, 0]",44.0,passed
5,"[0, 1, 0, 1, 0, 1, 0, 1, 0]",40.0,passed
6,"[0, 1, 0, 1, 0, 0, 1, 1, 0]",36.0,passed
7,"[0, 1, 0, 1, 0, 0, 0, 0, 0]",23.0,passed
8,"[0, 1, 0, 1, 0, 0, 0, 1, 1]",43.0,passed


New best found!
Iteration:  1
Current best:  [0, 1, 0, 1, 1, 0, 0, 1, 0] 	  Obj func res:  44.0


,vector,obj_res,constraints
0,"[1, 1, 0, 1, 1, 0, 0, 1, 0]",52.0,passed
1,"[0, 0, 0, 1, 1, 0, 0, 1, 0]",33.0,passed
2,"[0, 1, 1, 1, 1, 0, 0, 1, 0]",53.0,passed
3,"[0, 1, 0, 0, 1, 0, 0, 1, 0]",32.0,passed
4,"[0, 1, 0, 1, 0, 0, 0, 1, 0]",30.0,passed
5,"[0, 1, 0, 1, 1, 1, 0, 1, 0]",54.0,passed
6,"[0, 1, 0, 1, 1, 0, 1, 1, 0]",50.0,passed
7,"[0, 1, 0, 1, 1, 0, 0, 0, 0]",37.0,passed
8,"[0, 1, 0, 1, 1, 0, 0, 1, 1]",57.0,passed


New best found!
Iteration:  2
Current best:  [0, 1, 0, 1, 1, 0, 0, 1, 1] 	  Obj func res:  57.0


,vector,obj_res,constraints
0,"[1, 1, 0, 1, 1, 0, 0, 1, 1]",65.0,passed
1,"[0, 0, 0, 1, 1, 0, 0, 1, 1]",46.0,passed
2,"[0, 1, 1, 1, 1, 0, 0, 1, 1]",66.0,failed
3,"[0, 1, 0, 0, 1, 0, 0, 1, 1]",45.0,passed
4,"[0, 1, 0, 1, 0, 0, 0, 1, 1]",43.0,passed
5,"[0, 1, 0, 1, 1, 1, 0, 1, 1]",67.0,failed
6,"[0, 1, 0, 1, 1, 0, 1, 1, 1]",63.0,passed
7,"[0, 1, 0, 1, 1, 0, 0, 0, 1]",50.0,passed
8,"[0, 1, 0, 1, 1, 0, 0, 1, 0]",44.0,passed


New best found!
Iteration:  3
Current best:  [1, 1, 0, 1, 1, 0, 0, 1, 1] 	  Obj func res:  65.0


,vector,obj_res,constraints
0,"[0, 1, 0, 1, 1, 0, 0, 1, 1]",57.0,passed
1,"[1, 0, 0, 1, 1, 0, 0, 1, 1]",54.0,passed
2,"[1, 1, 1, 1, 1, 0, 0, 1, 1]",74.0,failed
3,"[1, 1, 0, 0, 1, 0, 0, 1, 1]",53.0,passed
4,"[1, 1, 0, 1, 0, 0, 0, 1, 1]",51.0,passed
5,"[1, 1, 0, 1, 1, 1, 0, 1, 1]",75.0,failed
6,"[1, 1, 0, 1, 1, 0, 1, 1, 1]",71.0,failed
7,"[1, 1, 0, 1, 1, 0, 0, 0, 1]",58.0,passed
8,"[1, 1, 0, 1, 1, 0, 0, 1, 0]",52.0,passed


New neighborhood doesn't contain improvement, optimum found!
Optimum:  [1, 1, 0, 1, 1, 0, 0, 1, 1] 	  Obj func res:  65.0


([1, 1, 0, 1, 1, 0, 0, 1, 1], 65.0, 5)

# Genetic Algorithm

Genetic algorithms (GAs) are a popular optimization algorithm, falling under the domain of Evolutionary Programming. GAs are population based metaheurisitic. A randomised initial population of n members goes through an iterative process which results in an optimum. Each iteration the population undergoes evolution inspired changes causing the population to converge to an optimum. The evolutionary operators employed for a GA are: 
- selection (with bias toward high fitness individuals ie survival of the fitest), 
- cross-over of two parents to produce two off-spring
- randomised mutation of off-spring's genes.   

The GA is used below to solve a typical travelling salesmen optimization problem. Here the decision variables and decision vector represents a permuation vector.

In [8]:
 distance_matrix = np.array([
[0, 32, 39, 42, 29, 35],
[32, 0, 36, 27, 41, 25],
[39, 36, 0, 28, 33, 40],
[42, 27, 28, 0, 27, 38],
[29, 41, 33, 27, 0, 26],
[35, 25, 40, 38, 26, 0]])

In [9]:
s_0 = [1,2,3,4,5,6] # this is a permutation vector, it describes the order in which the cities are visited. The last city is the first, so the distance from the last to first entry must be added at the end.

## Optimization problem functions and GA helper functions

In [10]:
# objective function for the tsp problem is the sum of the distances travelled. 
def objective(x):
    vect_len = len(x) # num cities
    city_distances = [] # empty list to store distances that must be summed

    # loop through each city pair and extract distance from the distance matrix
    for i in range(vect_len):
        # print(i)
        # find matrix indices for distances between two sucessive cities in the permutation vector
        city_a = x[i]-1 # -1 because pythons indexing starts at 0

        # next city is either i+1 in the permutation vector, or if i is the final entry the next city is first in the permutation vector
        if i < vect_len-1:
            city_b = x[i+1]-1
        else:
            city_b = x[0]-1

        # get distance now that matrix indices are  known
        dist = distance_matrix[int(city_a),int(city_b)]
        # print('[',city_a+1,',',city_b+1,']','\t',dist)

        # save single city distance
        city_distances.append(dist)

    return sum(city_distances)

# this function initialises a population of n permutation vectors
def initialise_population(num_cities, n):

    population = [] # empty list to store population

    # randomly scramble this vector n times
    for i in range(n):

        # initially gen individual permutation vectors as 1 to num_cities
        individual = np.array([i+1 for i in range(num_cities)])
        np.random.shuffle(individual) # randomly shuffle this permutation vector

        # store this individual into the population
        population.append(individual)

    return population

# function to evaluate fitness of each individual in a current population
def evaluate_fitness(population):
    pop_fitness = pd.DataFrame(columns=['individual','fitness'])

    # pass each individual in population to objective function to evaluate their fitness
    for individual in population:
        s = objective(individual)
        # print(individual,'\t',s)

        # store results into dataframre
        pop_fitness = pop_fitness.append({'individual':individual,'fitness':s},ignore_index=True)

    # fix column datatype issues
    pop_fitness['fitness'] = pop_fitness['fitness'].astype(float)

    # display population with their fitness
    display(pop_fitness)

    return pop_fitness

# function to perform tournament select on population: results in three pairs of parents
def selection(population,num_parent_pairs):

    population_temp = population.copy()
    parents = pd.DataFrame(columns=['individual','fitness','pairs']) # empty dataframe to store parents
    pair_indicator = 0 # counter used to identify parent pairs

    # loop through population and perform tournament selection
    print('Performing tournament selection')
    for i in range(num_parent_pairs*2):

        # determine fitness weighted probability for each individual
        population_temp['probability'] = 1/population_temp['fitness']

        # randomly select three individuals for the tournament and weight the samply probability based on their fitness reciperocal ie Prob = 1/objective_function(s)
        tournament = population_temp.sample(n=3,weights='probability')

        # tournament winner is indivial with min fitness
        winner_index = tournament['fitness'].idxmin()
        winner_vector = population_temp['individual'].loc[winner_index] 
        winner_fitness = population_temp['fitness'].loc[winner_index]
        
        # save winer to parents dataframe
        if i % 2 == 0:
            pair_indicator += 1 

        parents = parents.append({'individual':winner_vector,'fitness':winner_fitness,'pairs':pair_indicator},ignore_index=True)
        
        # remove winner from population
        population_temp.drop(index=winner_index,inplace=True)
        population_temp.reset_index(inplace=True,drop=True)

    # display parents
    print('Parents')
    display(parents)

    return parents

# function to perform cross over on parent population, results in six new off spring
def crossover(parents,num_parent_pairs):

    # dataframe to hold children
    children = pd.DataFrame(columns=['individual'])

    # perform crossover for each parent pair
    for i in range(1,num_parent_pairs+1):

        # select two parents from parents dataframe
        parent_1 = parents[parents['pairs']==i].iloc[0]['individual']
        parent_2 = parents[parents['pairs']==i].iloc[1]['individual']

        # perform two point crossover at 3rd and 5th positions
        child_1 = np.zeros(6,dtype=int)
        child_2 = np.zeros(6,dtype=int)

        # cross over for child 1
        child_1[0:2] = parent_1[0:2]
        child_1[5] = parent_1[5]
        c = list(parent_2.copy())
        [ c.remove(int(i)) for i in (set(child_1) - set([0])) ] # this looks which genes from parents two arent in child 1 yet
        child_1[2:5] = c # this populates child 1 with parent two's genes in the order those genes occur in parent two

        # cross over for child 2
        child_2[0:2] = parent_2[0:2]
        child_2[5] = parent_2[5]
        c = list(parent_1.copy())
        [ c.remove(int(i)) for i in (set(child_2) - set([0])) ] # this looks which genes from parents two arent in child 1 yet
        child_2[2:5] = c # this populates child 1 with parent two's genes in the order those genes occur in parent two

        # save children
        children = children.append({'individual':child_1,'pairs':i},ignore_index=True)
        children = children.append({'individual':child_2,'pairs':i},ignore_index=True)
    print('Children:')
    display(children)

    return children

# function to mutate children, this function random mutates of two children for each children pair. The mutation randomly swaps two genes in the chosen child.
def mutation(children,num_parent_pairs):

    # add column to track which children are mutated
    children['mutated'] = np.zeros(6)

    # loop through each offspring pair and mutate as needed
    for i in range(1,num_parent_pairs+1):
        # re-insert list of lists into dataframe after a child is mutated. This it a fix for a funny with Pandas dataframes with entries which are lists
        list_of_children = [children['individual'].iloc[i] for i in range(6)]

        # which child to mutate and which genes to swap
        who_to_mutate = random.randint(0,1)
        which_genes_to_swap = random.sample([i for i in range(6)],2) # randomly select two integers in range 1 to 6.

        # subset children df to look at off spring pairs ons
        child_index = children.index[children['pairs']==i][who_to_mutate]
        child_to_mutate = children.loc[children['pairs']==i]['individual'].iloc[who_to_mutate] # individual permutation vector
        # print('Child to mutate: ', child_to_mutate)

        # mutate child by swapping two genes
        swap_1_temp = child_to_mutate[which_genes_to_swap[0]] 
        swap_2_temp = child_to_mutate[which_genes_to_swap[1]] 

        child_to_mutate[which_genes_to_swap[0]] = swap_2_temp
        child_to_mutate[which_genes_to_swap[1]] = swap_1_temp

        # insert mutated child in place of original
        list_of_children[child_index] = child_to_mutate
        children['individual']  = list_of_children
        children.loc[child_index,'mutated'] = 1
        # print('Mutated positions: ',which_genes_to_swap)
        # print('Mutated child: ',children['individual'].iloc[child_index])


    print('Mutated children')
    display(children)

    return children    

## GA implementation

In [12]:
def genetic_algorithm(init_population):

    # step 1: initialise iteration and best known solution
    iteration = 0
    s_star = []
    f_s_star = 10000000 # large initial value for f(s_star)

    # find best solution from initial population
    print('Init population fitness')
    population = evaluate_fitness(init_population)
    
    # store first best individual and their fitness
    fittest_individ_index  = population['fitness'].idxmin()
    s_star = population.loc[fittest_individ_index]['individual']
    f_s_star = population.loc[fittest_individ_index]['fitness']

    print('Best starting individual: \n',s_star,f_s_star)

    # loop through successive iterations
    stopping_criteria = False # this will be set to true, to terminate while loop, when 8 generations have passed with no improvement in the incumbent
    incumbent_fitness = pd.DataFrame(columns=['Generation','Incumbent','Fitness'])
    incumbent_fitness = incumbent_fitness.append({'Generation':iteration,'Incumbent':s_star,'Fitness':f_s_star},ignore_index=True)


    while (iteration <= 40) and (stopping_criteria == False):
        print('\n***************************************************************************************************')
        print('Generation :', iteration)

        print('Population: ')
        display(population)
        
        # print population mean fitness
        mean_fitness = population['fitness'].mean()
        print('Population mean fitness: ', mean_fitness)

        # step 2: Selection process via Tournament select
        parents =  selection(population,num_parent_pairs=3)

        # step 3: Crossover to generate offspring
        children = crossover(parents,num_parent_pairs=3)

        # step 4: mutation of a random child in children pair takes place
        mutated_children = mutation(children,num_parent_pairs=3)

        # step 5 evaluate children fitness
        list_of_children = [mutated_children['individual'].iloc[i] for i in range(6)]
        print('Children fitness:')
        children_fitness = evaluate_fitness(list_of_children)
        
        # step 6 combine parents and children
        combined = parents[['individual','fitness']].append(children_fitness[['individual','fitness']])

        # step 7 new generation, sort combined by fitness ascending, new gen is BEST 8 individuals
        combined.sort_values(by=['fitness'],ascending=True,inplace=True,ignore_index=True)

        # new generation is top 8 invididuals by fitness from combined current population and offspring
        new_population = [combined['individual'].iloc[i] for i in range(8)]
        population = combined.iloc[0:8,:]

        # step 8 check if new population contains better BEST individual
        if combined['fitness'][0] < f_s_star:
            s_star = combined['individual'][0]
            f_s_star = combined['fitness'][0]
            print('New best individual: \n',s_star,f_s_star)

        # step 9 increment generation counter
        iteration += 1

        # store incumebent information
        incumbent_fitness = incumbent_fitness.append({'Generation':iteration,'Incumbent':s_star,'Fitness':f_s_star},ignore_index=True)

        # check if stopping criteria met
        if iteration > 8:
            # get a set of distinct fitness values for incumbent of last 8 generations
            last_8_generations_fitness =  set(incumbent_fitness['Fitness'].iloc[-8:]) # set will only allow unique values, so if there only a single value then all last 8 generations have the same fitness

            # if length of set == 1 stopping condition is met
            if len(last_8_generations_fitness) == 1:
                stopping_criteria = True
                print('Eight generation have elapsed without an improvement in the incumbent!')
                print('Found optimum: ', s_star, ' Objective function value: ', f_s_star)

# test algo    
init_population = initialise_population(num_cities=6, n=8)
genetic_algorithm(init_population)


Init population fitness


,individual,fitness
0,"[2, 5, 3, 4, 1, 6]",204.0
1,"[4, 5, 3, 2, 1, 6]",201.0
2,"[6, 2, 1, 3, 4, 5]",177.0
3,"[3, 1, 6, 2, 5, 4]",195.0
4,"[4, 2, 5, 6, 1, 3]",196.0
5,"[2, 5, 6, 1, 4, 3]",208.0
6,"[4, 6, 3, 5, 2, 1]",226.0
7,"[1, 2, 3, 6, 5, 4]",203.0


Best starting individual: 
 [6 2 1 3 4 5] 177.0

***************************************************************************************************
Generation : 0
Population: 


,individual,fitness
0,"[2, 5, 3, 4, 1, 6]",204.0
1,"[4, 5, 3, 2, 1, 6]",201.0
2,"[6, 2, 1, 3, 4, 5]",177.0
3,"[3, 1, 6, 2, 5, 4]",195.0
4,"[4, 2, 5, 6, 1, 3]",196.0
5,"[2, 5, 6, 1, 4, 3]",208.0
6,"[4, 6, 3, 5, 2, 1]",226.0
7,"[1, 2, 3, 6, 5, 4]",203.0


Population mean fitness:  201.25
Performing tournament selection
Parents


...

Mutated children


,individual,pairs,mutated
0,"[6, 2, 4, 5, 1, 3]",1.0,1.0
1,"[6, 5, 4, 1, 3, 2]",1.0,0.0
2,"[6, 3, 1, 2, 4, 5]",2.0,1.0
3,"[6, 5, 4, 3, 1, 2]",2.0,0.0
4,"[6, 1, 4, 3, 5, 2]",3.0,1.0
5,"[6, 2, 1, 3, 4, 5]",3.0,0.0


Children fitness:


,individual,fitness
0,"[6, 2, 4, 5, 1, 3]",187.0
1,"[6, 5, 4, 1, 3, 2]",195.0
2,"[6, 3, 1, 2, 4, 5]",191.0
3,"[6, 5, 4, 3, 1, 2]",177.0
4,"[6, 1, 4, 3, 5, 2]",204.0
5,"[6, 2, 1, 3, 4, 5]",177.0


Eight generation have elapsed without an improvement in the incumbent!
Found optimum:  [6 2 4 3 1 5]  Objective function value:  174.0


# Fitness landscape exploration for a knapsack problem

When presented with an optimization problem, the solution or fitness space is typical not known. Have some idea or intuition of this space - are there many local minima / maxima, are the large plains, are there saddle points etc - goes a long way to aiding the appropriate selection of an optimization algorithm.

In [13]:
def random_local_search(x):
    s = x # initial decision vector 
    f_s = objective(s) # initial decision vector objection function value
    iteration = 0 # iteration counter
    max_iterations = 99 # stopping criteria = max number of iterations

    # dataframe to store random walk
    df_random_walk = pd.DataFrame(columns=['iteration','vector','obj_res'])
    df_random_walk = df_random_walk.append({'iteration':iteration,'vector':s,'obj_res':f_s},ignore_index=True)

    while iteration < max_iterations:

        print('Iteration: ', iteration, ' Current best: ',s,'\t',' Obj func res: ',f_s)

        # step 1: generate new neighbours
        new_neighbours = single_bit_complementing_operator(s)

        # step 2: evaluate neighbours objective function values + check constraints
        neighbour_results = neighbour_selection(new_neighbours)

        # step 3: select random neighbour
        feasible_neighbours = neighbour_results.loc[neighbour_results['constraints']=='passed'] # only consider neighbours who satisfy both constraints

        # select random neighbour
        random_int = random.randint(0,len(feasible_neighbours)-1)
        random_new_neighbour = feasible_neighbours['vector'].iloc[random_int]
        random_new_neighbour_obj_res = feasible_neighbours['obj_res'].iloc[random_int]


        # step 5: increment iterator
        iteration += 1
        
        # store new random neigbours
        s = random_new_neighbour
        f_s = random_new_neighbour_obj_res

        df_random_walk = df_random_walk.append({'iteration':iteration,'vector':random_new_neighbour,'obj_res':random_new_neighbour_obj_res},ignore_index=True)


    return df_random_walk


# Initial starting conditions to test

s_0_1 = [0, 1, 1, 1, 0, 0, 1, 1, 1]
s_0_2 = [0, 1, 0, 1, 0, 0, 0, 1, 0]

# test algorithm
df_random_walk = random_local_search(s_0_1)


Iteration:  0  Current best:  [0, 1, 1, 1, 0, 0, 1, 1, 1] 	  Obj func res:  140
Iteration:  1  Current best:  [0, 0, 1, 1, 0, 0, 1, 1, 1] 	  Obj func res:  140.0
Iteration:  2  Current best:  [0, 0, 1, 1, 0, 0, 0, 1, 1] 	  Obj func res:  140.0
Iteration:  3  Current best:  [0, 0, 1, 1, 0, 0, 1, 1, 1] 	  Obj func res:  140.0
Iteration:  4  Current best:  [0, 0, 1, 1, 0, 0, 1, 1, 0] 	  Obj func res:  140.0
Iteration:  5  Current best:  [0, 1, 1, 1, 0, 0, 1, 1, 0] 	  Obj func res:  140.0
Iteration:  6  Current best:  [0, 1, 1, 1, 0, 0, 0, 1, 0] 	  Obj func res:  140.0
Iteration:  7  Current best:  [1, 1, 1, 1, 0, 0, 0, 1, 0] 	  Obj func res:  140.0
Iteration:  8  Current best:  [1, 1, 1, 1, 1, 0, 0, 1, 0] 	  Obj func res:  140.0
Iteration:  9  Current best:  [1, 0, 1, 1, 1, 0, 0, 1, 0] 	  Obj func res:  210.0
Iteration:  10  Current best:  [1, 0, 1, 1, 1, 0, 0, 0, 0] 	  Obj func res:  140.0
Iteration:  11  Current best:  [1, 0, 1, 0, 1, 0, 0, 0, 0] 	  Obj func res:  210.0
Iteration:  12  

## sub question a)

In [15]:
from scipy.spatial import distance
# calculating random walk correlation function
variance = df_random_walk['obj_res'].var()
mean = df_random_walk['obj_res'].mean()

m = 100
d = 1

sumation = 0
for i in range(m-d):
    sumation += (df_random_walk['obj_res'][i]-mean)*(df_random_walk['obj_res'][i+1]-mean)

r_1 = (1/((m-d)*variance))*sumation
print('Random walk correlation function: ',r_1)

# calculating correlation length
l = 1/abs(np.log(abs(r_1)))
print('Correlation lenght: ',l) 

# determine pairwise hamming distance because we use bit vectors
max_distance = 0
vect1_pos = 0
vect2_pos = 0
for i in range(100):
    for j in range(100):
        vector_1 = df_random_walk['vector'].iloc[i]
        vector_2 = df_random_walk['vector'].iloc[j]
        dist_ij = distance.hamming(vector_1, vector_2)*len(vector_1)

        if dist_ij > max_distance:
            max_distance = dist_ij
            vect1_pos = i
            vect2_pos = j

print('Max distance: ',max_distance, ' between vectors: ',vect1_pos,vect2_pos)

# normalised correlation length
normalised_l = l/max_distance
print('Normalised correlation length: ',normalised_l) 


Random walk correlation function:  0.608272138228941
Correlation lenght:  2.011534535759817
Max distance:  9.0  between vectors:  6 73
Normalised correlation length:  0.22350383730664636


## sub-question b)

In [16]:
# determine candidate solutions with only two items
store = []
counter = 1
df_subset = pd.DataFrame(columns=['vector','obj_res'])
for i in range(9):
    new_list = np.array([0,0,0,0,0,0,0,0,0])
    new_list[i] = 1
    for j in range(i+1,9):
        new_new_list = new_list.copy()
        new_new_list[j] = 1
        new_new_list_obj_res = objective(new_new_list)
        df_subset = df_subset.append({'vector':new_new_list,'obj_res':new_new_list_obj_res},ignore_index=True)

df_subset

,vector,obj_res
0,"[1, 1, 0, 0, 0, 0, 0, 0, 0]",70
1,"[1, 0, 1, 0, 0, 0, 0, 0, 0]",140
2,"[1, 0, 0, 1, 0, 0, 0, 0, 0]",140
3,"[1, 0, 0, 0, 1, 0, 0, 0, 0]",140
4,"[1, 0, 0, 0, 0, 1, 0, 0, 0]",140
5,"[1, 0, 0, 0, 0, 0, 1, 0, 0]",140
6,"[1, 0, 0, 0, 0, 0, 0, 1, 0]",140
7,"[1, 0, 0, 0, 0, 0, 0, 0, 1]",70
8,"[0, 1, 1, 0, 0, 0, 0, 0, 0]",70
9,"[0, 1, 0, 1, 0, 0, 0, 0, 0]",140


In [18]:
# run local search algorithm for each of these two item candidate solutions

# dataframe to store results
df_results =  pd.DataFrame(columns=['Initial','initial_obj_res','Local Optimum','final_obj_res','length_descent_walk'])

for i in range(len(df_subset)):
    # get initial starting postion
    initial_vector = df_subset['vector'].iloc[i]
    initial_obj_res = df_subset['obj_res'].iloc[i]

    # call local search algo
    s,f_s,len_descent_walk = local_search(initial_vector)

    # save results
    df_results = df_results.append({'Initial':initial_vector,'initial_obj_res':initial_obj_res,'Local Optimum':s,'final_obj_res':f_s,'length_descent_walk':len_descent_walk},ignore_index=True)

display(df_results)

,Initial,initial_obj_res,Local Optimum,final_obj_res,length_descent_walk
0,"[1, 1, 0, 0, 0, 0, 0, 0, 0]",70,"[1, 1, 0, 1, 1, 1, 1, 0, 1]",74.0,7
1,"[1, 0, 1, 0, 0, 0, 0, 0, 0]",140,"[1, 1, 1, 1, 1, 0, 1, 0, 1]",73.0,7
2,"[1, 0, 0, 1, 0, 0, 0, 0, 0]",140,"[1, 1, 0, 1, 1, 1, 1, 0, 1]",74.0,7
3,"[1, 0, 0, 0, 1, 0, 0, 0, 0]",140,"[1, 1, 0, 1, 1, 1, 1, 0, 1]",74.0,7
4,"[1, 0, 0, 0, 0, 1, 0, 0, 0]",140,"[1, 1, 0, 1, 1, 1, 1, 0, 1]",74.0,7
5,"[1, 0, 0, 0, 0, 0, 1, 0, 0]",140,"[1, 1, 0, 1, 1, 1, 1, 0, 1]",74.0,7
6,"[1, 0, 0, 0, 0, 0, 0, 1, 0]",140,"[1, 1, 0, 1, 1, 0, 0, 1, 1]",65.0,6
7,"[1, 0, 0, 0, 0, 0, 0, 0, 1]",70,"[1, 1, 0, 1, 1, 1, 1, 0, 1]",74.0,7
8,"[0, 1, 1, 0, 0, 0, 0, 0, 0]",70,"[1, 1, 1, 1, 1, 0, 1, 0, 1]",73.0,7
9,"[0, 1, 0, 1, 0, 0, 0, 0, 0]",140,"[1, 1, 0, 1, 1, 1, 1, 0, 1]",74.0,7


In [19]:
# calculate the average length of descent walks
llm = df_results['length_descent_walk'].sum()/len(df_results)
print('Average length of descent walks: ',llm )

Average length of descent walks:  6.694444444444445


## sub question c)

In [20]:
# calculate amplitude of U
amp_u = 9*(df_results['initial_obj_res'].max()-df_results['initial_obj_res'].min())/df_results['initial_obj_res'].sum()

# Calculate amplitude of O
amp_o = 9*(df_results['final_obj_res'].max()-df_results['final_obj_res'].min())/df_results['final_obj_res'].sum()

# calculate delta O
delta_o = (amp_u - amp_o) / amp_u

print('Relative variation: ',delta_o)

# calculate average gap(o)
gap_o = sum(df_results['final_obj_res'].max()-df_results['final_obj_res'])/(9*df_results['final_obj_res'].max())

print('Average gap: ',gap_o)

Relative variation:  0.45710928319623967
Average gap:  0.16666666666666666
